# Logit模型

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import pandas as pd
import statsmodels.api as sm # 使用 statsmodels 包实现 Logit 模型

In [ ]:
# 设置随机种子以确保结果可重现
np.random.seed(42)

# 生成自变量：学习时间（小时）
n_samples = 50  # 样本量
study_hours_new = np.random.uniform(1, 8, n_samples)

# 生成线性预测值
true_beta0 = -3.5  # 真实截距
true_beta1 = 0.8   # 真实斜率
linear_pred = true_beta0 + true_beta1 * study_hours_new

# 计算通过概率
pass_prob = 1 / (1 + np.exp(-linear_pred))

# 生成二元因变量：是否通过考试
pass_exam_new = np.random.binomial(1, pass_prob) # 生成服从二项分布的随机数的函数

# 创建DataFrame
df_logit = pd.DataFrame({
    'study_hours': study_hours_new,
    'pass_exam': pass_exam_new,
    'pass_prob': pass_prob
})

df_logit.head()

,study_hours,pass_exam,pass_prob
0,3.621781,1,0.353755
1,7.655000,1,0.932390
2,6.123958,0,0.802052
3,5.190609,0,0.657571
4,2.092130,0,0.138680


In [29]:
# 准备数据
X_new = sm.add_constant(study_hours_new)
y_new = pass_exam_new

# 拟合 Logit 模型
logit_model_new = sm.Logit(y_new, X_new)

# 使用不同方法拟合并比较结果
result_newton_new = logit_model_new.fit(method='newton', disp=0)
result_bfgs_new = logit_model_new.fit(method='bfgs', disp=0)

# # 输出两种方法的结果比较
# print("\n=== Newton方法结果 ===")
# print(result_newton_new.summary())

# print("\n=== BFGS方法结果 ===")
# print(result_bfgs_new.summary())

# 输出两种方法的结果比较
print("\n=== Newton方法结果 ===")
print(f"beta0 = {result_newton_new.params[0]:.4f}")
print(f"beta1 = {result_newton_new.params[1]:.4f}")
print(f"最大对数似然值 = {result_newton_new.llf:.4f}")

print("\n=== BFGS方法结果 ===")
print(f"beta0 = {result_bfgs_new.params[0]:.4f}")
print(f"beta1 = {result_bfgs_new.params[1]:.4f}")
print(f"最大对数似然值 = {result_bfgs_new.llf:.4f}")

# 比较参数差异
print("\n=== 参数差异 ===")
print(f"beta0差异: {abs(result_newton_new.params[0] - result_bfgs_new.params[0]):.6f}")
print(f"beta1差异: {abs(result_newton_new.params[1] - result_bfgs_new.params[1]):.6f}")
print(f"对数似然差异: {abs(result_newton_new.llf - result_bfgs_new.llf):.6f}")


=== Newton方法结果 ===
beta0 = -4.1170
beta1 = 0.8870
最大对数似然值 = -22.5005

=== BFGS方法结果 ===
beta0 = -4.1171
beta1 = 0.8870
最大对数似然值 = -22.5005

=== 参数差异 ===
beta0差异: 0.000046
beta1差异: 0.000008
对数似然差异: 0.000000


- **模型：**
  $$P(y_i=1|x_i) = \frac{e^{\beta_0+\beta_1 x_i}}{1+e^{\beta_0+\beta_1 x_i}}$$

- **似然函数：**
  $$L(\beta_0,\beta_1) = \prod_{i=1}^{50} \left[\frac{e^{\beta_0+\beta_1 x_i}}{1+e^{\beta_0+\beta_1 x_i}}\right]^{y_i}\left[\frac{1}{1+e^{\beta_0+\beta_1 x_i}}\right]^{1-y_i}$$

- **对数似然：**
  $$\ln L = \sum_{i=1}^{n} \left[ y_i \cdot \ln \left( \frac{e^{\beta_0 + \beta_1 x_i}}{1 + e^{\beta_0 + \beta_1 x_i}} \right) + (1 - y_i) \cdot \ln \left( \frac{1}{1 + e^{\beta_0 + \beta_1 x_i}} \right) \right]$$
  $$\ln L = \sum_{i=1}^{50} \{y_i(\beta_0+\beta_1 x_i) - \ln(1+e^{\beta_0+\beta_1 x_i})\}$$

- **梯度（一阶导数）：**
  $$\frac{\partial \ln L}{\partial \beta_0} = \sum_{i=1}^{50} \left[y_i - \frac{e^{\beta_0+\beta_1 x_i}}{1+e^{\beta_0+\beta_1 x_i}}\right] = \sum_{i=1}^{50} (y_i - p_i)$$
  
  $$\frac{\partial \ln L}{\partial \beta_1} = \sum_{i=1}^{50} \left[y_i x_i - \frac{e^{\beta_0+\beta_1 x_i}}{1+e^{\beta_0+\beta_1 x_i}}x_i\right] = \sum_{i=1}^{50} (y_i - p_i)x_i$$

  其中 $p_i = \frac{e^{\beta_0+\beta_1 x_i}}{1+e^{\beta_0+\beta_1 x_i}}$ 是第 $i$ 个观测的预测概率。

In [35]:
# 优化的Logit模型对数似然函数（向量化实现）
def neg_log_likelihood_optimized(params, x, y):
    beta0, beta1 = params
    linear_pred = beta0 + beta1 * x
    
    # 直接按照对数似然公式计算: Σ[y_i(β₀+β₁x_i) - ln(1+e^(β₀+β₁x_i))]
    log_likelihood = np.sum(y * linear_pred - np.log(1 + np.exp(linear_pred)))
    # print(f"参数: beta0={beta0:.4f}, beta1={beta1:.4f}, 对数似然: {log_likelihood:.4f}")
    return -log_likelihood

# 计算梯度以加速优化
def neg_log_likelihood_grad(params, x, y):
    beta0, beta1 = params
    linear_pred = beta0 + beta1 * x
    p = 1 / (1 + np.exp(-linear_pred))  # 计算预测概率
    # 梯度计算
    grad_beta0 = -np.sum(y - p)
    grad_beta1 = -np.sum((y - p) * x)
    return np.array([grad_beta0, grad_beta1])

# 使用DataFrame中的数据
x = df_logit['study_hours'].values
y = df_logit['pass_exam'].values

# 初始猜测值
initial_guess = [0, 0]


result_optimized = optimize.minimize(
    neg_log_likelihood_optimized, 
    initial_guess, 
    args=(x, y),
    method='L-BFGS-B',
    jac=neg_log_likelihood_grad,
    options={'disp': True}
)

# 提取最优参数
beta0_opt, beta1_opt = result_optimized.x
print("\n优化方法最终结果:")
print(f"beta0 = {beta0_opt:.4f}")
print(f"beta1 = {beta1_opt:.4f}")
print(f"最大对数似然值 = {-result_optimized.fun:.4f}")
print(f"收敛状态: {result_optimized.success}")
print(f"迭代次数: {result_optimized.nit}")

# 比较与真实参数的差异
print("\n与真实参数比较:")
print(f"beta0: 真实值={true_beta0:.4f}, 估计值={beta0_opt:.4f}, 差异={abs(true_beta0-beta0_opt):.4f}")
print(f"beta1: 真实值={true_beta1:.4f}, 估计值={beta1_opt:.4f}, 差异={abs(true_beta1-beta1_opt):.4f}")




优化方法最终结果:
beta0 = -4.1171
beta1 = 0.8870
最大对数似然值 = -22.5005
收敛状态: True
迭代次数: 12

与真实参数比较:
beta0: 真实值=-3.5000, 估计值=-4.1171, 差异=0.6171
beta1: 真实值=0.8000, 估计值=0.8870, 差异=0.0870
